In [23]:
import requests
import json
import numpy as np
import pandas as pd

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import rasterio as rt
from rasterio.mask import mask

%matplotlib inline
import matplotlib.pyplot as plt

import plotly.graph_objects as go



In [ ]:
"""
Looking on how to choose the .tif file desirede for a given address
"""

In [ ]:
# look at 3D houses code page for the choice of the .tif file --- HARV .. surface, bounceries..

In [1]:
import shapefile
shape = shapefile.Reader("data/DHMVIIDSMRAS1m_k01/DHMVII_vdc_k01.shp")
#first feature of the shapefile
feature = shape.shapeRecords()[0]
first = feature.shape.__geo_interface__  
print(first) # (GeoJSON format)
{'type': 'LineString', 'coordinates': ((0.0, 0.0), (25.0, 10.0), (50.0, 50.0))}

{'type': 'MultiPolygon', 'coordinates': [[[(162079.0100390613, 238604.76999878138), (162098.55007813126, 238604.11000274494), (162099.80984375626, 238605.04999755695), (162099.99985607713, 238609.5300009176), (162099.99992141128, 238605.86000366136), (162099.99991637468, 238600.73001953587), (162099.9999263212, 238599.4500073269), (162099.9999173805, 238583.15000427142), (162099.99991682172, 238575.62999267876), (162099.9999158308, 238561.35000121966), (162099.9999171421, 238561.21000976488), (162099.99992598593, 238560.4899987802), (162099.9999215901, 238553.46999633312), (162099.9999134466, 238530.71998901293), (162099.9999116361, 238523.87000549585), (162099.99991690367, 238518.23999877647), (162099.99992281944, 238515.9900006093), (162099.99992214888, 238515.15001220256), (162099.99991682172, 238507.37001343071), (162099.99991680682, 238507.3299969472), (162099.99991650134, 238496.28000152484), (162099.99991893023, 238494.0900146477), (162099.99992378056, 238490.51999755576), (1620

{'type': 'LineString', 'coordinates': ((0.0, 0.0), (25.0, 10.0), (50.0, 50.0))}

In [ ]:
"""
Collecting the necessary data for a 3D plot
"""

In [40]:
# google search like:  python geopandas what data is needed for a 3D plot





In [ ]:
# https://hatarilabs.com/ih-en/tutorial-to-convert-geospatial-data-shapefile-to-3d-data-vtk-with-python-geopandas-pyvista

In [1]:
# Create an array OR dataframe with the plot data: x, y from .tif and z form the CHM (Canopy Height Models)

In [40]:
import requests
import json
import numpy as np
import pandas as pd

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import rasterio as rt
from rasterio.mask import mask

%matplotlib inline
import matplotlib.pyplot as plt

import plotly.graph_objects as go

from typing import Dict 
    

In [41]:
address = "Lange Van Sterbeeckstraat 8, 2060 Antwerpen"

In [50]:

def get_address_data_from_geopunt(address: str):
    data = requests.get(f"https://loc.geopunt.be/v4/Location?q={address}").json()
#    print(data,"\n")
    info = {'address' : address, 
                'x_value' : data['LocationResult'][0]['Location']['X_Lambert72'],
                'y_value' : data['LocationResult'][0]['Location']['Y_Lambert72'],
                'street' : data['LocationResult'][0]['Thoroughfarename'],
                'house_number' : data['LocationResult'][0]['Housenumber'], 
                'postcode': data['LocationResult'][0]['Zipcode'], 
                'municipality' : data['LocationResult'][0]['Municipality']}
    
    detail = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch", 
                          params={"postcode": info['postcode'], 
                                  "straatnaam": info['street'],
                                  "huisnummer": info['house_number']}).json()
    building = requests.get(detail['adresMatches'][0]['adresseerbareObjecten'][0]['detail']).json()
    build = requests.get(building['gebouw']['detail']).json()
    info['polygon'] = [build['geometriePolygoon']['polygon']]
#    print(info['polygon'][0]['coordinates'][0])
#    print(info['polygon'][1]['coordinates'][1])
    
    
    return info

In [51]:
address_info = get_address_data_from_geopunt(address)
address_info

{'address': 'Lange Van Sterbeeckstraat 8, 2060 Antwerpen',
 'x_value': 153908.9,
 'y_value': 212660.38,
 'street': 'Lange Van Sterbeeckstraat',
 'house_number': '8',
 'postcode': '2060',
 'municipality': 'Antwerpen',
 'polygon': [{'coordinates': [[[153913.09697395563, 212652.07560559362],
     [153909.20398195833, 212669.40699760243],
     [153904.5820299536, 212668.39899760485],
     [153907.5011979565, 212656.54043759406],
     [153908.57345395535, 212652.1845335923],
     [153908.816653952, 212651.1965655908],
     [153909.12673395127, 212651.2602455914],
     [153913.09697395563, 212652.07560559362]]],
   'type': 'Polygon'}]}

In [59]:
import rioxarray
import geopandas as gpd
surface_HARV = rioxarray.open_rasterio('data/DHMVIIDSMRAS1m_k01/GeoTIFF/DHMVIIDSMRAS1m_k01.tif')
print(surface_HARV.min())
print(surface_HARV.max())
aoi_boundary_HARV = gpd.read_file("data/DHMVIIDSMRAS1m_k01/DHMVII_vdc_k01.shp")
print(aoi_boundary_HARV)

aoi_boundary_HARV.bounds


<xarray.DataArray ()>
array(-9999.)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray ()>
array(98.55999756)
Coordinates:
    spatial_ref  int64 0
   Id   OpnDatum1   OpnDatum2  \
0   0  2013-02-20        None   
1   0  2013-04-01        None   
2   0  2013-02-20  2013-04-01   

                                            geometry  
0  MULTIPOLYGON Z (((162079.010 238604.770 0.000,...  
1  POLYGON Z ((148803.300 243432.780 0.000, 14882...  
2  MULTIPOLYGON Z (((156554.470 243736.800 0.000,...  


,minx,miny,maxx,maxy
0,144900.0,237758.67,162100.030020,247224.389995
1,144900.0,238850.25,162100.000132,243457.759998
2,144900.0,238493.37,162100.000116,243768.959990


In [12]:
import pandas as pd


import numpy as np
import rasterio
from rasterio.plot import show

# Get DTM data
dtm_src = 'data/DHMVIIDTMRAS1m_k01/GeoTIFF/DHMVIIDTMRAS1m_k01.tif'
dtm_img = rasterio.open(dtm_src)
dtm_array = dtm_img.read()

# Get DSM data
dsm_src = 'data/DHMVIIDSMRAS1m_k01/GeoTIFF/DHMVIIDSMRAS1m_k01.tif'
dsm_img = rasterio.open(dsm_src)
#show(dsm_img)
dsm_array = dsm_img.read()

# Calculate the CHM (Canopy Height Models)
height_dsm_min_dtm = dsm_array - dtm_array

#height_list = height_dsm_min_dtm
#df_z = ()


#df_height = pd.DataFrame(height_dsm_min_dtm)

plot_data = address_coor


df_xy = pd.DataFrame(address_coor)



In [15]:
df_xy
#df_height
height_dsm_min_dtm

array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.02000046],
        [0.        , 0.        , 0.        , ..., 0.01000023,
         0.        , 0.01000023]]], dtype=float32)

In [111]:
# PLOT PLOT PLOT
from shapely.geometry import Polygon

xy = address_info["polygon"]

plot = np.array(xy)

x, y = plot.exterior.coords.xy

#ax.plot_surface(x, y, array)

#plot_points.plot(plot_points)
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'exterior'

In [70]:
# let's check if our address is in on the first .tif..
x = address_info["x_value"]
y = address_info["y_value"]
address = x, y
address

(153908.9, 212660.38)

In [80]:
map_x = surface_HARV.min()
map_y = surface_HARV.max()
filemap = map_x, map_y
filemap

(<xarray.DataArray ()>
 array(-9999.)
 Coordinates:
     spatial_ref  int64 0,
 <xarray.DataArray ()>
 array(98.55999756)
 Coordinates:
     spatial_ref  int64 0)

In [84]:
type(map_x)


xarray.core.dataarray.DataArray